In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import optuna

import dataProcess

from datetime import datetime

In [13]:
hourly = '.\\Raw data\\ten_year\\weatherstats_toronto_hourly.csv'
daily = '.\\Raw data\\ten_year\\weatherstats_toronto_daily.csv'
start = datetime(2014, 4, 16).date()
end = datetime(2024, 4, 10).date()
data = dataProcess.generateDataNoLoader(hourly, daily, start, end)
train_data = data['train']
val_data = data['validation']
test_data = data['test']

In [14]:
class FlatWeatherMLP(nn.Module):
    def __init__(self, input_size=312, hidden_size=150):
        super(FlatWeatherMLP, self).__init__()
        self.inputLayer = nn.Linear(input_size, hidden_size)
        self.hiddenLayer = nn.Linear(hidden_size, hidden_size)
        self.outputLayer = nn.Linear(hidden_size, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = x.to(torch.float32)
        x = torch.flatten(x, start_dim=1)
        x = self.inputLayer(x)
        x = self.relu(x)
        x = self.hiddenLayer(x)
        x = self.relu(x)
        x = self.outputLayer(x)
        x = torch.squeeze(x)
        return x

In [15]:
def accuracy(model, data):
    batch_size = 100
    totalDiff = 0.0
    count = 0
    loader = DataLoader(data, batch_size=batch_size)
    for data, target in loader:
        count = count + len(target)
        z = model(data)
        totalDiff += torch.sum(torch.abs(target - z))
    return(totalDiff / count)    


In [26]:

def train_model(model, trainData, valData, lr=.002, batchSize=10, epochs=100, v=True):
    loader = DataLoader(train_data, batch_size=batchSize, shuffle=True)
    vloader = DataLoader(val_data, batch_size=batchSize, shuffle=True)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    for epoch in range(epochs):
        totalLoss = 0
        c = 0
        for d, t in loader:
            t = torch.squeeze(t)
            c += 1
            if torch.isnan(d).any():
                print(d)
                print(c)
            optimizer.zero_grad()
            z = model(d)
            z = torch.squeeze(z)
            # print(z)
            # print(t)
            loss = criterion(z, t)
            loss.backward()
            optimizer.step()
            totalLoss += loss.item()
        if v:
            print(f'Epoch [{epoch+1}/{epochs}], Training Loss: {totalLoss / len(loader)}')
        model.eval()
        totalLossVal = 0
        with torch.no_grad():
            for d, t in vloader:
                z = model(d)
                loss = criterion(z, t)
                totalLossVal += loss.item()
        if v:
            print(f'Epoch [{epoch+1}/{epochs}], Validation Loss: {totalLossVal / len(loader)}')
    print(accuracy(model, data['validation']))
# model = FlatWeatherMLP()
# train_model(model, train_data, val_data)
# accuracy(model, data['validation'])

In [27]:
def objective(trial):
    lr = trial.suggest_float('lr', .001, .05)
    batchSize = trial.suggest_int('batchSize', 1, 50)
    epochs = trial.suggest_int('epochs', 5, 100)
    hidden_size = trial.suggest_int('hidden_size', 10, 312)
    model = FlatWeatherMLP(hidden_size=hidden_size)
    train_model(model, train_data, val_data, lr=lr, batchSize=batchSize, epochs=epochs, v=False)
    return accuracy(model, data['validation'])

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2024-04-17 15:24:12,182] A new study created in memory with name: no-name-604c2f91-8ea0-4ada-a32d-6a1bdfbc3d49
[I 2024-04-17 15:24:25,552] Trial 0 finished with value: 84.33026885986328 and parameters: {'lr': 0.027083513940992676, 'batchSize': 30, 'epochs': 35, 'hidden_size': 247}. Best is trial 0 with value: 84.33026885986328.


tensor(84.3303, grad_fn=<DivBackward0>)


[I 2024-04-17 15:24:54,723] Trial 1 finished with value: 3.6715922355651855 and parameters: {'lr': 0.008155626482340653, 'batchSize': 31, 'epochs': 78, 'hidden_size': 218}. Best is trial 1 with value: 3.6715922355651855.


tensor(3.6716, grad_fn=<DivBackward0>)


[I 2024-04-17 15:25:16,151] Trial 2 finished with value: 9.835444450378418 and parameters: {'lr': 0.01470019564482726, 'batchSize': 11, 'epochs': 81, 'hidden_size': 37}. Best is trial 1 with value: 3.6715922355651855.


...

tensor(9.3553, grad_fn=<DivBackward0>)


[I 2024-04-17 16:51:16,102] Trial 98 finished with value: 5.209029197692871 and parameters: {'lr': 0.0010372138275194789, 'batchSize': 28, 'epochs': 84, 'hidden_size': 115}. Best is trial 61 with value: 2.569736957550049.


tensor(5.2090, grad_fn=<DivBackward0>)


[I 2024-04-17 16:51:27,290] Trial 99 finished with value: 3.7644388675689697 and parameters: {'lr': 0.010679875969997392, 'batchSize': 24, 'epochs': 81, 'hidden_size': 18}. Best is trial 61 with value: 2.569736957550049.


tensor(3.7644, grad_fn=<DivBackward0>)


In [28]:
print(study.best_params)

model = FlatWeatherMLP(hidden_size=study.best_params['hidden_size'])
train_model(model, train_data, val_data, lr=study.best_params['lr'], batchSize=study.best_params['batchSize'], epochs=study.best_params['epochs'])
accuracy(model, data['test'])

{'lr': 0.004799821565505988, 'batchSize': 26, 'epochs': 84, 'hidden_size': 45}
Epoch [1/84], Training Loss: 71247.9309501648
Epoch [1/84], Validation Loss: 190.5688204084124
Epoch [2/84], Training Loss: 458.65963345482237
Epoch [2/84], Validation Loss: 89.21472967238654
Epoch [3/84], Training Loss: 217.2152556010655
Epoch [3/84], Validation Loss: 78.5974622453962
Epoch [4/84], Training Loss: 150.8791781380063
Epoch [4/84], Validation Loss: 62.77451969328381
Epoch [5/84], Training Loss: 138.26231002807617
Epoch [5/84], Validation Loss: 83.00271315801712
Epoch [6/84], Training Loss: 126.36934548332577
Epoch [6/84], Validation Loss: 91.23574447631836
Epoch [7/84], Training Loss: 114.38732183547248
Epoch [7/84], Validation Loss: 53.9630066099621
Epoch [8/84], Training Loss: 79.8512043271746
Epoch [8/84], Validation Loss: 33.9762597878774
Epoch [9/84], Training Loss: 133.08675721713476
Epoch [9/84], Validation Loss: 55.2186766124907
Epoch [10/84], Training Loss: 107.95678470248268
Epoch [10

tensor(3.0493, grad_fn=<DivBackward0>)

This linear model achieves an accuracy of an average of 3.05 degrees from the true results when predicting the next day's temperature using the current day's weather data